In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from helpers.funcs import *

In [2]:
config = json.load(open('../config.json', 'r'))
INPUT_DIR = config['INPUT_DIR']
TWEET_DIR = config['TWEET_DIR']
events = open(INPUT_DIR + 'event_names.txt', 'r').read().splitlines()

In [3]:
votes = pd.read_csv('input/2016_votes.csv')  # Rep voter share (among Dem and Rep votes)
abbr = votes.set_index('state')['abbr'].to_dict()

In [4]:
# data on these events have their location marked
has_loc = ['dallas',
 'baton_rouge',
 'burlington',
 'fort_lauderdale',
 'fresno',
 'san_francisco',
 'vegas',
 'thornton',
 'sutherland_springs',
 'parkland',
 'nashville',
 'santa_fe',
 'annapolis']

In [5]:
total_partisan = {}
rep_twitter = {}
for i, e in enumerate(has_loc):
    print(e)
    tweets = pd.read_csv(TWEET_DIR +e+'/'+e+'.csv', sep='\t', lineterminator='\n', usecols=['remove', 'isRT', 'user_id', 'dem_follows', 'rep_follows', 'geo_state'])
    tweets = tweets[~tweets['remove']] 
    #tweets = tweets[~tweets['isRT']] # uncomment to ignore retweets
    tweets['user_id'] = tweets['user_id'].astype(int)
    tweets['geo_state'] = tweets['geo_state'].astype(str)
    tweets['geo_state_abbr'] = tweets['geo_state'].map(abbr)
    state_known = tweets[~tweets['geo_state_abbr'].isnull()]
    prop_marked = len(state_known) / len(tweets)
    states = state_known.groupby('geo_state_abbr')
    for state, group in states:
        group = group.drop_duplicates('user_id', keep='first') # deduplicate users
        dem, rep = split_party(group)
        total = len(dem) + len(rep)
        if state not in total_partisan:
            total_partisan[state] = total
            rep_twitter[state] = len(rep)
        else:
            total_partisan[state] += total
            rep_twitter[state] += len(rep)

dallas


/anaconda2/envs/nlu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


baton_rouge
burlington
fort_lauderdale
fresno
san_francisco
vegas
thornton
sutherland_springs
parkland
nashville
santa_fe
annapolis


In [8]:
sum(total_partisan.values())

344634

In [13]:
for k, v in rep_twitter.items():
    rep_twitter[k] = v / total_partisan[k]

In [14]:
votes['partisan_twitter'] = votes['abbr'].map(total_partisan)
votes['rep_twitter'] = votes['abbr'].map(rep_twitter)

In [15]:
votes.to_csv('output/verify_party_assignment.csv', index=False)
votes

,state,abbr,num_voters,dem,rep,rep_share,partisan_twitter,rep_twitter
0,Alabama,AL,2123372,34.36%,62.08%,0.643716,3630,0.655923
1,Alaska,AK,318608,36.55%,51.28%,0.583855,649,0.565485
2,Arizona,AZ,2604657,44.58%,48.08%,0.518886,6811,0.510057
3,Arkansas,AR,1130635,33.65%,60.57%,0.642857,1973,0.608718
4,California,CA,14237893,61.48%,31.49%,0.338711,46615,0.268712
5,Colorado,CO,2780247,48.16%,43.25%,0.473143,6712,0.390048
6,Connecticut,CT,1644920,54.57%,40.93%,0.428586,3443,0.275922
7,Delaware,DE,443814,53.09%,41.71%,0.439979,577,0.322357
8,"Washington, D.C.",DC,311268,90.86%,4.09%,0.043075,10252,0.306282
9,Florida,FL,9502747,47.41%,48.6%,0.506197,23707,0.513140
